# Fitting is an Art!

Python macro for testing which fitting procedure is likely to give the "best" results.

In this case, we consider **Gaussian distributions on constant background (peak fitting and searching)**, where the fit precision and hypothesis testing can be improved by sharing common parameters.

### Authors:
- Troels Petersen ([email](mailto:petersen@nbi.dk))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from iminuit import Minuit, cost
from scipy import stats

In [ ]:
r = np.random
r.seed(42)
SavePlots = False

## CASE: Gaussian distributions on a constant background:

The initial fitting function is the following:

* $f_{1}(x) = \text{Gauss + Constant} =  \frac{N_{sig}}{\sigma\sqrt(2\pi)}\cdot \exp \left[-0.5 \cdot\left(\frac{(x-\mu)}{\sigma}\right)^{2} \right] + C~~~$ for $x$ in $[-\infty,\infty]$

It disregards that there are additional signal peaks at higher values. Your job is to expand the fit until it really describes the data.

In [ ]:
# Signal parameters:
Npoints_gauss = 2000
fN     = 0.5
mux    = 1.43
dmux   = 1.41
sigmax = 0.15          

# Background parameters:
Npoints_pol0 = 5000

# Binning parameters:
xmin   =  0.0
xmax   = 10.0
Nbins  =  200
binwidth_gauss = (xmax-xmin) / float(Nbins)
print(f"  The bin width is: {binwidth_gauss:5.2f}")

In [ ]:
# Fill histogram with signal and background events:
Npeak = np.random.geometric(fN, Npoints_gauss)                  # Make random assignment to Gaussian peaks
sig = np.random.normal(loc=mux+dmux*(Npeak-1), scale=sigmax)    # Now generate signal according to these peaks
bkg = np.random.uniform(xmin, xmax, size=Npoints_pol0)

data = np.concatenate([sig, bkg])
counts, bin_edges = np.histogram(data, bins=Nbins, range=(xmin, xmax))
unc_count = np.sqrt(counts)
x = bin_edges[:-1]+(bin_edges[1]-bin_edges[0])/2.

fig, ax = plt.subplots(figsize=(16, 8))
ax.errorbar(x, counts, yerr=unc_count, marker = 'o', ls='')
ax.set_xlabel('x', fontsize=18)
ax.set_ylabel('Frequency / 0.05', fontsize=18);

### Define function (including bin width to get normalisation right):


In [ ]:
def func_gpol0(x, N, mu, sigma, cst) :
    norm = binwidth_gauss * N / np.sqrt(2.0*np.pi) / sigma
    z = (x-mu)/sigma
    return norm * np.exp(-0.5*z*z) + cst

In [ ]:
mask = counts>0
cfit = cost.LeastSquares(x[mask], counts[mask], unc_count[mask], func_gpol0)
mfit = Minuit(cfit, N=1.0, mu=0.0, sigma=1.0, cst=1.0)
mfit.migrad()

In [ ]:
if (not mfit.fmin.is_valid) :                                   # Check if the fit converged
    print("  WARNING: The ChiSquare fit DID NOT converge!!!")

ax.plot(x, func_gpol0(x, *mfit.values[:]), 'r', linewidth=2.0, label='Const + Gauss fit')

# Adding fit results to plot:
chi2 = mfit.fval                     # ChiSquare value
Ndof = len(x[mask]) - mfit.nfit      # Number of (non-empty) bins
Prob = stats.chi2.sf(chi2, Ndof)     # ChiSquare probability given Ndof

fit_info = [f"$\\chi^2$ / $N_\\mathrm{{dof}}$ = {chi2:.1f} / {Ndof}", f"Prob($\\chi^2$, $N_\\mathrm{{dof}}$) = {Prob:.3f}",]
for p, v, e in zip(mfit.parameters, mfit.values[:], mfit.errors[:]) :
    Ndecimals = max(0,-np.int32(np.log10(e)-1-np.log10(2)))                                # Number of significant digits
    fit_info.append(f"{p} = ${v:{10}.{Ndecimals}{"f"}} \\pm {e:{10}.{Ndecimals}{"f"}}$")

ax.legend(title="\n".join(fit_info), fontsize=18, title_fontsize = 18, alignment = 'center');
fig

# Questions:

 1. Look at the data plot and the corresponding fit. What type of fit is it? Does it
    run well (or at all)? And once it runs, does it seem to be reasonable? Why/why not?
    What is the p-value from the minimal Chi2 that your fit obtained, once you got any
    "reasonable" fit to work? Is it acceptable?
    Does the fitting function include all features of the data? Why/why not? Try for
    2-5 minutes and discuss it with others (or just think for yourself), before reading on!

---
_2-5 minutes later_...

---

 2. As it happens, there seem to be a additional peaks. Try to write a new and
    expandeded fitting function, which includes these features in the model, and get the
    fit to run. How significant is the additional peak(s), based on significance of the
    amplitude? And what test would you apply to this, if you wanted to make a full-fledged
    hypothesis test between the two models? Are they nested? Can you actually get a number out?
    Again, discuss it before coding on.

---
_10-20 minutes later_...

---

 3. Imagine that you concluded that there were additional new peaks, and that you were sure that
    they had the same width as the original peak (for example because the width was due to
    the resolution of the apperatus, and not the peak itself). Does that help you in the fit,
    and if so, how? Does the significance of the peaks increase? Would it always do that?
    Also imagine, that the parameter of interest is the distance between the peaks. How
    would you now write your fitting function?

 4. How would you test, if the peaks are really equidistant? Do so...

 5. Assuming the peaks to be equidistant, how would you test, if the peaks are consistent with an unobserved peak at exactly zero? Do so...

 6. If one wanted to test the G+pol0 vs. the N*G+pol0 models against each other, which might be relevant, then considering the difference in ChiSquare values or -2ln of the likelihood ratio would be obvious (these two gives the same result, when errors are Gaussian and the binning does not have any effect). Wilk's theorem would provide the way to produce a p-value, thus doing a proper hypothesis test using the likelihood ratio test:

* Using iminuit, fit the data with both hypothesis, and note the Chi2 or LLH value (using `mfit.fval`).
* Then compute the test statistic $\chi^2_{1} - \chi^2_{2}$ or $-2\log{\frac{LH_{1}}{LH_{2}}}$, and see that it is $\chi^{2}$ distributed (Wilk's Theorem) from repeating many experiments.

NOTE: The test statistic distribution will depend on how many peaks you fit and if you (smartly) eliminated one parameter from the second fit.

# Learning points:

This exercise in "fitting tricks" should teach you that:
1. __Good starting values is paramount!__ (Almost all fits fail with poor starting values).
2. The form of the fitting function is also important.<br>
   a. Ensure that the x-values do not represent some small range far from 0.<br>
   b. Ensure that you give the fitting function enough freedom to fit the data.<br>
   c. Conversely, try to curb the number of parameters, if there are arguments for doing so (calibration peaks).<br>
   d. Make sure that you've normalised your fitting PDFs, to avoid correlations between normalisation and parameters.
3. If a fit continues to fail, try simply to draw the function and starting values on top of the data. Often, they don't match well (general advice, not just in this exercise).